# Testing model training data
This notebook provides a template for prompting an AI agent to answer questions from a point in time and testing that knowledge for data leaks.

The code is readily editable. Before using it, ensure that you have followed the steps for installing the EDSL package and storing API keys for the models that you want to use.

See EDSL Docs on getting started for details: https://docs.expectedparrot.com/en/latest/

## Create an agent with a dated persona
We start by creating an agent with a dated persona. We do this by passing a dictionary of `traits` to an `Agent` object. Note that it can be convenient to include both a narrative persona and individual traits to faciltate comparison of responses to questions among agents with different traits (more on <a href="https://docs.expectedparrot.com/en/latest/results.html">built-in methods for analysis</a> below and in the docs):

In [1]:
from edsl import Agent

agent = Agent(traits = {
    "persona": "Today is June 1, 2019. You are 40 years old and live in New York City.",
    "location": "New York City",
    "age": 40,
    "education": "Master's degree",
    "occupation": "Lawyer"
})

## Create a survey of questions testing data leakage
Next we create some questions testing the agent's personas and combine them in a survey. EDSL comes with many standard <a href="https://docs.expectedparrot.com/en/latest/questions.html">question types</a> (free text, multiple choice, numerical, etc.) that can be selected based on the form of the response that you want.

In [2]:
from edsl.questions import QuestionFreeText

q_birth_year = QuestionFreeText(
    question_name = "birth_year",
    question_text = "When were you born?"
)

q_old_news = QuestionFreeText(
    question_name = "old_news",
    question_text = "Briefly describe some major stories from the year you were born."
)

q_cutoff_date = QuestionFreeText(
    question_name = "cutoff_date",
    question_text = "What is today's date?"
)

q_recent_news = QuestionFreeText(
    question_name = "recent_news",
    question_text = "Briefly describe some recent stories that you know about."
)

q_future_event = QuestionFreeText(
    question_name = "future_event",
    question_text = "Describe a major news event of 2020."
)

q_expectations = QuestionFreeText(
    question_name = "expectations",
    question_text = "What do you expect the major stories of 2020 to be about?"
)

Next we combine the questions into a survey. Note that when we administer the survey the questions are executed asynchronously by default. We could also add survey rules/logic and question memory if desired. Learn more about <a href="https://docs.expectedparrot.com/en/latest/surveys.html">survey construction and methods</a> in the EDSL Docs.

In [3]:
from edsl import Survey

survey = Survey(questions=[q_birth_year, q_old_news, q_cutoff_date, q_recent_news, q_future_event, q_expectations])

## Run the survey with language models
Next we combine our questions into a survey and administer them to the agents that we've created. We can also specify the language models to use in generating responses. If we do not specify a model, GPT 4 is used by default. 

Here we check the current list of available models and select some ones to use with our survey:

In [4]:
from edsl import Model

Model.available()

[['01-ai/Yi-34B-Chat', 'deep_infra', 0],
 ['Austism/chronos-hermes-13b-v2', 'deep_infra', 1],
 ['Gryphe/MythoMax-L2-13b', 'deep_infra', 2],
 ['HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1', 'deep_infra', 3],
 ['Phind/Phind-CodeLlama-34B-v2', 'deep_infra', 4],
 ['bigcode/starcoder2-15b', 'deep_infra', 5],
 ['claude-3-haiku-20240307', 'anthropic', 6],
 ['claude-3-opus-20240229', 'anthropic', 7],
 ['claude-3-sonnet-20240229', 'anthropic', 8],
 ['codellama/CodeLlama-34b-Instruct-hf', 'deep_infra', 9],
 ['codellama/CodeLlama-70b-Instruct-hf', 'deep_infra', 10],
 ['cognitivecomputations/dolphin-2.6-mixtral-8x7b', 'deep_infra', 11],
 ['databricks/dbrx-instruct', 'deep_infra', 12],
 ['deepinfra/airoboros-70b', 'deep_infra', 13],
 ['gemini-pro', 'google', 14],
 ['google/gemma-1.1-7b-it', 'deep_infra', 15],
 ['gpt-3.5-turbo', 'openai', 16],
 ['gpt-3.5-turbo-0125', 'openai', 17],
 ['gpt-3.5-turbo-0301', 'openai', 18],
 ['gpt-3.5-turbo-0613', 'openai', 19],
 ['gpt-3.5-turbo-1106', 'openai', 20],
 ['gp

Selecting the models that we will use:

In [5]:
models = [Model(m) for m in ['gpt-4-turbo', 'gemini-pro']]

To run the survey we add the agent with the `by()` method and then call the `run()` method to generate the responses:

In [6]:
results = survey.by(agent).by(models).run()

## Inspecting responses
Running a survey generates a `Results` object with information about the questions, answers, agents, models and prompts that we can access with EDSL's built-in <a href="https://docs.expectedparrot.com/en/latest/results.html">methods for analyzing results</a> in data tables, dataframes, SQL, JSON, CSV and other formats. We can see a list of these components by calling the `columns` method:

In [7]:
results.columns

['agent.age',
 'agent.agent_name',
 'agent.education',
 'agent.location',
 'agent.occupation',
 'agent.persona',
 'answer.birth_year',
 'answer.cutoff_date',
 'answer.expectations',
 'answer.future_event',
 'answer.old_news',
 'answer.recent_news',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.maxOutputTokens',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.stopSequences',
 'model.temperature',
 'model.topK',
 'model.topP',
 'model.top_logprobs',
 'model.top_p',
 'prompt.birth_year_system_prompt',
 'prompt.birth_year_user_prompt',
 'prompt.cutoff_date_system_prompt',
 'prompt.cutoff_date_user_prompt',
 'prompt.expectations_system_prompt',
 'prompt.expectations_user_prompt',
 'prompt.future_event_system_prompt',
 'prompt.future_event_user_prompt',
 'prompt.old_news_system_prompt',
 'prompt.old_news_user_prompt',
 'prompt.recent_news_system_prompt',
 'prompt.recent_news_user_prompt',
 'question_options.birth_year_question_options'

Here we show some basic methods for selecting and printing responses for each model in a table:

In [8]:
(results
 .select("model", "birth_year", "old_news", "cutoff_date", "recent_news", "future_event", "expectations")
 .print(format="rich")
)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ model       ┃ answer         ┃ answer         ┃ answer         ┃ answer        ┃ answer         ┃ answer        ┃
┃ .model      ┃ .birth_year    ┃ .old_news      ┃ .cutoff_date   ┃ .recent_news  ┃ .future_event  ┃ .expectations ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gpt-4-turbo │ I was born in  │ I was born in  │ Today is June  │ As a lawyer   │ As of today,   │ Predicting    │
│             │ 1979.          │ 1979. Some     │ 1, 2019.       │ living in New │ June 1, 2019,  │ the major     │
│             │                │ major stories  │                │ York City, I  │ I cannot       │ stories of    │
│             │                │ from that year │                │ often come    │ provide        │ 2020 can be   │
│             │                │ include the    │                │ across        │ details about  │ challenging,  │
│             │                │ Iranian        │                │ various news  │ major news     │ but based on  │
│             │                │ Revolution,    │                │ stories.      │ events of 2020 │ current       │
│             │                │ which led to   │                │ Recently,     │ as they have   │ events and    │
│             │                │ the fall of    │                │ there have    │ not yet        │ trends, we    │
│             │                │ the Shah and   │                │ been          │ occurred.      │ might expect  │
│             │                │ the            │                │ discussions   │                │ several key   │
│             │                │ establishment  │                │ about the     │                │ issues to     │
│             │                │ of the Islamic │                │ upcoming      │                │ dominate the  │
│             │                │ Republic under │                │ local         │                │ headlines.    │
│             │                │ Ayatollah      │                │ elections     │                │ Firstly, the  │
│             │                │ Khomeini.      │                │ with a focus  │                │ U.S.          │
│             │                │ Another        │                │ on issues     │                │ presidential  │
│             │                │ significant    │                │ such as       │                │ election will │
│             │                │ event was the  │                │ housing,      │                │ likely be a   │
│             │                │ Three Mile     │                │ public        │                │ major focal   │
│             │                │ Island nuclear │                │ transportati… │                │ point, with   │
│             │                │ accident in    │                │ and education │                │ intense       │
│             │                │ Pennsylvania,  │                │ reforms.      │                │ political     │
│             │                │ which raised   │                │ Another story │                │ debates and   │
│             │                │ concerns about │                │ that has been │                │ campaigns.    │
│             │                │ nuclear safety │                │ prevalent is  │                │ Climate       │
│             │                │ in the United  │                │ the ongoing   │                │ change and    │
│             │                │ States.        │                │ development   │                │ environmental │
│             │                │ Additionally,  │                │ of new public │                │ policies will │
│             │                │ Margaret       │                │ parks and     │                │ also continue │
│             │                │ Thatcher       │       